In [ ]:
from sentence_transformers import SentenceTransformer, util

## Steps to compute F1-Embedding:

In [ ]:
# precision: how many matched keyphrases with the ground-truth in the output 
# recall: how many matched keyphrases in the output w.r.t all ground-truth keyphrases

#todo:- get word embedding from a pre-trained model (e.g. BERT)
# compute semantic similarity match between two words if cosine_sim > 5.0 ==> consider as a matching

In [4]:
## Evaluation with test data

groundtruth_keyphrases= [
'complex biological processes',
'robust model-order reduction',
'high dimensional nonlinear partial differential equation model',
'nonlinear distributed parameter model',
'pilot-scale BNR activated sludge plant',
'state-space model',
'singular perturbation approximation balanced truncating technique',
'modelling errors',
'controller design',
'Hankel singular values',
'biological nutrient removal activated sludge processes',
'biotechnology',
'control system synthesis',
'nonlinear control systems',
'nonlinear differential equations',
'partial differential equations',
'reduced order systems',
'robust control',
'singularly perturbed systems',
'state-space methods',
'water treatment'

]


predicted_keyphrases=[
'model order reduction', 
'reduction complex biological', 
'biological process based', 
'model complex biological', 
'pde model complex', 
'model process', 
'order reduction complex',
'reduce original model', 
'pde model presented', 
'biological processes'
'organic_process', 
'life_processes',
'life_function',
'biological_process', 
'process_(biological)',
'physiological_process'
]


In [ ]:
# read files by the id (fname)- pair reading from the two folders.
with open('ground-truth file path') as fileIn:
    groundtruth_keyphrases = fileIn.readlines()
    
    fileIn.close()
    
with open('predicted-keyphrases file path') as fileIn:
    predicted_keyphrases = fileIn.readlines()
    
    fileIn.close()

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
# # init embedding
# embedding = TransformerDocumentEmbeddings('bert-base-uncased')

groundtruth_embedding= model.encode(groundtruth_keyphrases, convert_to_tensor=True)
keyphrase_embedding = model.encode(predicted_keyphrases, convert_to_tensor=True)


#Compute cosine-similarits
cosine_score = util.pytorch_cos_sim(keyphrase_embedding, groundtruth_embedding)

In [34]:
num_of_Similar=0 # number of similar keyphrases

for similarty_score in cosine_score:
    if any(similarity_threshould > 0.5 for similarity_threshould in similarty_score):
        num_of_Similar+=1
        
recall= num_of_Similar/len(groundtruth_keyphrases)
precision= num_of_Similar/len(predicted_keyphrases)

F1= 2*(recall*precision)/(recall+precision)

print ('number of similar terms: ',num_of_Similar)
print ('Recall: ',"%.3f"%recall)
print('Precision: ', "%.3f"%precision)
print ('F1: ', "%.3f"%F1)

number of similar terms:  14
Recall:  0.667
Precision:  0.933
F1:  0.778
